# Mask debugging

In [1]:
# %load_ext autoreload
# %autoreload 2
# from pipelines import build_pipeline, build_coco_pipeline
# import tensorflow as tf
# import re
# import numpy as np
# import matplotlib.pyplot as plt
# import time
# import datetime
# import textwrap
# import json
# import pandas as pd
# import os
# import pickle
# # Tokenizers
# from tokenizers import KerasTokenizer
# # Model
# from models.CaptioningTransformers import CaptioningTransformer
# from models.Callbacks import CaptioningCallback
# from models.model_utils import load_captioning_model
# # Results
# from prediction import Predict
# from tqdm import tqdm
# from evaluation import Evaluate
# from models.OldCaptioningTransformer import instantiate_captioner
# import seaborn as sns

In [2]:
from experiment import Experiment, AblationStudy, ExperimentCluster
config_1={
    'name':'model_big',
    'dataset':'medpix',
    'model_name':'exp_3',
    'seq_len':178,
    'num_heads':8,
    'model_dim':512,
    'dropout':0.2,
    'img_backbone_trainable':False,
    'img_size':(299,299,3),
    'img_backbone':'efficientnetb0',
    'intermediate_size':512*4,
    'vocab_size':None,
    'capts_per_img':1,
    'num_encoder_blocks':6,
    'num_decoder_blocks':6,
    'max_epochs':2,
    'learning_rate':0.001,
    'normalization':'pre',
    'metric':'val_acc'
}

config_2={
    'name':'model_mid',
    'dataset':'medpix',
    'model_name':'exp_3',
    'seq_len':178,
    'num_heads':4,
    'model_dim':256,
    'dropout':0.2,
    'img_backbone_trainable':False,
    'img_size':(299,299,3),
    'img_backbone':'efficientnetb0',
    'intermediate_size':256*4,
    'vocab_size':None,
    'capts_per_img':1,
    'num_encoder_blocks':3,
    'num_decoder_blocks':3,
    'max_epochs':2,
    'learning_rate':0.001,
    'normalization':'pre',
    'metric':'val_acc'
}


config_3={
    'name':'model_small',
    'dataset':'medpix',
    'model_name':'exp_3',
    'seq_len':178,
    'num_heads':1,
    'model_dim':128,
    'dropout':0.2,
    'img_backbone_trainable':False,
    'img_size':(299,299,3),
    'img_backbone':'efficientnetb0',
    'intermediate_size':128*4,
    'vocab_size':None,
    'capts_per_img':1,
    'num_encoder_blocks':1,
    'num_decoder_blocks':1,
    'max_epochs':2,
    'learning_rate':0.001,
    'normalization':'pre',
    'metric':'val_acc'
}

configs=[config_1,config_2,config_3]
experiment_cluster=ExperimentCluster(configs,'../models/cluster_1')
experiment_cluster.run()


/home/jupyter/MedClip/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running experiment with the following configuration:
{'name': 'model_big', 'dataset': 'medpix', 'model_name': 'exp_3', 'seq_len': 178, 'num_heads': 8, 'model_dim': 512, 'dropout': 0.2, 'img_backbone_trainable': False, 'img_size': (299, 299, 3), 'img_backbone': 'efficientnetb0', 'intermediate_size': 2048, 'vocab_size': None, 'capts_per_img': 1, 'num_encoder_blocks': 6, 'num_decoder_blocks': 6, 'max_epochs': 2, 'learning_rate': 0.001, 'normalization': 'pre', 'metric': 'val_acc'}
Loading data pipeline...


2023-02-08 21:08:57.359928: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 21:08:57.371179: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 21:08:57.372975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 21:08:57.375356: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Initializing tokenizer...
Tokenizing data...
Instantiating callbacks...
Creatng and training model...
Epoch 1/2


2023-02-08 21:09:23.827869: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


...

Initializing tokenizer...
Tokenizing data...
Instantiating callbacks...
Creatng and training model...
Epoch 1/2


2023-02-08 21:11:35.848502: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


1/1 [==============================] - 15s 15s/step - loss: 9.3072 - acc: 0.0000e+00 - val_loss: 9.1460 - val_acc: 0.0819
Epoch 2/2
1/1 [==============================] - 2s 2s/step - loss: 9.0627 - acc: 0.1068 - val_loss: 9.0013 - val_acc: 0.1589
Plotting learning curves...
Making predictions...


100%|██████████| 1/1 [00:01<00:00,  1.39s/it]


Evaluating...
Clearing GPU memory


In [3]:
experiment_cluster.results

{'name': ['model_big', 'model_mid', 'model_small'],
 'dataset': ['medpix', 'medpix', 'medpix'],
 'model_name': ['exp_3', 'exp_3', 'exp_3'],
 'seq_len': [178, 178, 178],
 'num_heads': [8, 4, 1],
 'model_dim': [512, 256, 128],
 'dropout': [0.2, 0.2, 0.2],
 'img_backbone_trainable': [False, False, False],
 'img_size': [(299, 299, 3), (299, 299, 3), (299, 299, 3)],
 'img_backbone': ['efficientnetb0', 'efficientnetb0', 'efficientnetb0'],
 'intermediate_size': [2048, 1024, 512],
 'vocab_size': [11099, 11099, 11099],
 'capts_per_img': [1, 1, 1],
 'num_encoder_blocks': [6, 3, 1],
 'num_decoder_blocks': [6, 3, 1],
 'max_epochs': [2, 2, 2],
 'learning_rate': [0.001, 0.001, 0.001],
 'normalization': ['pre', 'pre', 'pre'],
 'metric': ['val_acc', 'val_acc', 'val_acc'],
 'core_modality_f1_w': [0.0, 0.0, 0.0],
 'plane_f1_w': [0.0, 0.0, 0.0],
 'anatomy_f1_w': [1.0, 0.0, 0.0],
 'bleu': [9.176834880063286, 1.5401555084618097, 3.3747902377517707],
 'mp1': [100.0, 100.0, 60.0],
 'mp2': [90.0, 83.333333333